In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

# If cuda device exist, use that
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



In [3]:
num_epochs = 15
num_classes = 10
batch_size = 128
learning_rate = 0.001

In [19]:
train_dataset = torchvision.datasets.MNIST(root='../../data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='../../data/', train=False, transform=transforms.ToTensor())

train_loader=torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [16]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dense1 = nn.Linear(12 * 12 * 64, 128)
        self.dense2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, 0.25)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.dense1(x))
        x = F.dropout(x, 0.5)
        x = self.dense2(x)
        
        return F.log_softmax(x, dim=1)


In [17]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch [{} /{}], step [{} /{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
            
        

Epoch [1 /15], step [100 /469], Loss: 0.3781
Epoch [1 /15], step [200 /469], Loss: 0.1335
Epoch [1 /15], step [300 /469], Loss: 0.3456
Epoch [1 /15], step [400 /469], Loss: 0.0988
Epoch [2 /15], step [100 /469], Loss: 0.0483
Epoch [2 /15], step [200 /469], Loss: 0.0725
Epoch [2 /15], step [300 /469], Loss: 0.0441
Epoch [2 /15], step [400 /469], Loss: 0.0413
Epoch [3 /15], step [100 /469], Loss: 0.0592
Epoch [3 /15], step [200 /469], Loss: 0.0559
Epoch [3 /15], step [300 /469], Loss: 0.1108
Epoch [3 /15], step [400 /469], Loss: 0.0588
Epoch [4 /15], step [100 /469], Loss: 0.0509
Epoch [4 /15], step [200 /469], Loss: 0.0547
Epoch [4 /15], step [300 /469], Loss: 0.0307
Epoch [4 /15], step [400 /469], Loss: 0.0508
Epoch [5 /15], step [100 /469], Loss: 0.0517
Epoch [5 /15], step [200 /469], Loss: 0.0372
Epoch [5 /15], step [300 /469], Loss: 0.0489
Epoch [5 /15], step [400 /469], Loss: 0.0494
Epoch [6 /15], step [100 /469], Loss: 0.0298
Epoch [6 /15], step [200 /469], Loss: 0.0327
Epoch [6 /

In [22]:
from sklearn.metrics import roc_auc_score

preds = []
y_true = []

model.eval() # Set model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        detached_pred = predicted.detach().cpu().numpy()
        detached_label = labels.detach().cpu().numpy()
        
        for f in range(0, len(detached_pred)):
            preds.append(detached_pred[f])
            y_true.append(detached_label[f])
            
    print('Test Accuracy of the model on the 10000 test images: {:.2%}'.format(correct / total))
        
    preds = np.eye(num_classes)[preds]
    y_true = np.eye(num_classes)[y_true]
    auc = roc_auc_score(preds, y_true)
    print("AUC: {:.2%}".format(auc))
        
# Save the model checkpoint
torch.save(model.state_dict(), 'pytorch_mnist_cnn.ckpt')

Test Accuracy of the model on the 10000 test images: 98.69%
AUC: 99.27%
